In [2]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt

import panel as pn
import plotly.express as px
from pathlib import Path
from dotenv import load_dotenv

# Initialize the Panel Extensions (for Plotly)
pn.extension('plotly')
import hvplot.pandas

In [3]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("MAPBOX_API_KEY")
px.set_mapbox_access_token(mapbox_token)

In [4]:
# Import the CSVs to Pandas DataFrames
file_path = Path("sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

coordinates_path = Path("neighborhoods_coordinates.csv")
coordinates_data = pd.read_csv(coordinates_path)

In [5]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    "(I used plotly because matlibplot was not loading correctly onto tabs)"
    # YOUR CODE HERE!
    housing_units = sfo_data.groupby('year').mean()
    housing_units_mean = housing_units.drop(['sale_price_sqr_foot', 'gross_rent'], axis=1)

    std = housing_units_mean["housing_units"].std()
    min = housing_units_mean["housing_units"].min()-std
    max = housing_units_mean["housing_units"].max()+std


    housing_units_per_year_plot = px.bar(
    housing_units_mean,
    x = housing_units_mean.index,
    y = "housing_units",
    range_y = [min,max],
    labels = {"year":"Year","housing_units":"Housing Units"},
    title = "Housing Units in San Francisco from 2010 to 2016",
    width = 500, 
    height = 600
    )

    return housing_units_per_year_plot


def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    # YOUR CODE HERE!
    housing_units = sfo_data.groupby('year').mean()
    
    average_gross_rent_sqr_foot = housing_units[
    ['sale_price_sqr_foot', 'gross_rent']
    ]
    
    avg_gross_rent = average_gross_rent_sqr_foot['gross_rent']
    avg_gross_rent_plot = avg_gross_rent.hvplot(
    xlabel = 'Year',
    ylabel = 'Gross Rent',
    title = "Average Gross Rent in San Francisco",
    width=400
    )
    
    return avg_gross_rent_plot
    
def average_sales_price():
    """Average Sales Price Per Year."""
    
    # YOUR CODE HERE!
    housing_units = sfo_data.groupby('year').mean()
    average_gross_rent_sqr_foot = housing_units[
    ['sale_price_sqr_foot', 'gross_rent']
    ]
    
    avg_sale_price_sqr_foot = average_gross_rent_sqr_foot['sale_price_sqr_foot']
    avg_sale_price_sqr_foot_plot = avg_sale_price_sqr_foot.hvplot(
    xlabel = 'Year',
    ylabel = 'Avg. Sale Price',
    title = 'Average Sale Price per Square Foot in San Francisco',
    width=500
    )
    
    return avg_sale_price_sqr_foot_plot


def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    # YOUR CODE HERE!
    neighborhood_data_avg = sfo_data.groupby(['year', 'neighborhood']).mean()
    neighborhood_data_avg.reset_index(inplace=True)
    
    neighborhood_data_avg_plot = neighborhood_data_avg.hvplot.line(
    x = 'year',
    y = "sale_price_sqr_foot",
    ylabel = 'Avg. Sale Price per Square Foot',
    groupby = 'neighborhood'
    )
    
    return neighborhood_data_avg_plot


def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    # YOUR CODE HERE!
    most_expensive_neighborhood_avg = sfo_data.groupby('neighborhood').mean()
    most_expensive_neighborhood_avg = most_expensive_neighborhood_avg.sort_values('sale_price_sqr_foot', ascending=False)
    most_expensive_neighborhood_avg_top_10 = most_expensive_neighborhood_avg.head(10)
    most_expensive_neighborhood_avg_top_10.reset_index(inplace=True)
    
    top_10_neighborhood_plot = most_expensive_neighborhood_avg_top_10.hvplot.bar(
    xlabel = 'Neighborhood',
    ylabel = 'Avg. Sale Price per Square Foot',
    x = 'neighborhood',
    y = 'sale_price_sqr_foot',
    rot=90,
    title = 'Top 10 Most Expensive Neighborhoods in San Francisco',
    height = 400
    )
    
    return top_10_neighborhood_plot


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    # YOUR CODE HERE!
    most_expensive_neighborhood_avg = sfo_data.groupby('neighborhood').mean()
    most_expensive_neighborhood_avg = most_expensive_neighborhood_avg.sort_values('sale_price_sqr_foot', ascending=False)
    most_expensive_neighborhood_avg_top_10 = most_expensive_neighborhood_avg.head(10)
    most_expensive_neighborhood_avg_top_10.reset_index(inplace=True)
    
    top_10_neighborhood_plot = px.parallel_coordinates(most_expensive_neighborhood_avg_top_10, color='sale_price_sqr_foot')
    
    return top_10_neighborhood_plot


def parallel_categories():
    """Parallel Categories Plot."""
    
    # YOUR CODE HERE!
    most_expensive_neighborhood_avg = sfo_data.groupby('neighborhood').mean()
    most_expensive_neighborhood_avg = most_expensive_neighborhood_avg.sort_values('sale_price_sqr_foot', ascending=False)
    most_expensive_neighborhood_avg_top_10 = most_expensive_neighborhood_avg.head(10)
    most_expensive_neighborhood_avg_top_10.reset_index(inplace=True)
    
    top_10_neighborhood_plot = px.parallel_categories(most_expensive_neighborhood_avg_top_10, color='sale_price_sqr_foot')
    
    return top_10_neighborhood_plot

def neighborhood_map():
    """Neighborhood Map"""
    
    # YOUR CODE HERE!
    coordinates_data.rename(columns={"Neighborhood" : "neighborhood"}, inplace=True)
    
    neighborhood_mean = sfo_data.groupby('neighborhood').mean()
    neighborhood_mean.reset_index(inplace=True)
    
    combined_neighborhood_data = pd.merge(
    coordinates_data, neighborhood_mean, on="neighborhood")
    
    map_1 = px.scatter_mapbox(
    combined_neighborhood_data,
    lat = "Lat",
    lon = "Lon",
    color = "gross_rent",
    size = "sale_price_sqr_foot",
    title="Average Sale Price per Square Foot and Gross Rent in San Francisco",
    zoom = 11
    )
    
    return map_1

In [6]:
#putting in a MD tab with a brief intro to the plot. 
info = pn.pane.Markdown(
"""
This Dashboard presents a visual analysis of historical prices of house units, sale price per square foot and gross rent in
San Francisco, California from 2010 to 2016. You can navigate through the tabs above to explore more details about the
evolution of the real estate market on The Golden City across these years

The dashboard has 5 tabs as follows:  
1) Welcome - Containing short message providing context on the dashobaord data.  
2) Rental Analysis - 2010 to 2016, containting the follwoing charts:  
    - Housing Units Per Year <-- I used plotly because matlibplot was not loading correctly onto tabs 
    - Average Gross Rent Per Year  
    - Average Sales Price Per Year  
3) Neighborhood Analysis - 2010 to 2016, containting the follwoing charts:  
    - Average Prices by Neighborhood  
    - Top 10 Most Expensive Neighborhoods  
4) Parallel, containting the follwoing charts:  
    - Average values per neighborhood - Parallel Coordinates  
    - Average values per neighborhood - Parallel Categories   
5) Geospatial Analysis, containting an interactive map with data on average prices per neighborhood.   
"""
)

#4 tabs with data as per the rental analysis notebook.
tab_1_columns = pn.Column(
    ("# San Francisco Housing Rental Analysis - 2010 to 2016"),
    ("## Housing Units Per Year"),
    housing_units_per_year,
    ("## Average Gross Rent Per Year"),
    average_gross_rent,
    ("## Average Sales Price Per Year"),
    average_sales_price
)

tab_2_columns = pn.Column(
    ("# San Francisco Neighborhood Analysis - 2010 to 2016"),
    ("## Average Prices by Neighborhood"),
    average_price_by_neighborhood,
    ("## Top 10 Most Expensive Neighborhoods"),
    top_most_expensive_neighborhoods
)

tab_3_columns = pn.Column(
    ("# Parallel Coordinates and Parallel Categories Analysis"),
    ("## Parallel Coordinates Plot"),
    parallel_coordinates,
    ("## Parallel Categories Plot"),
    parallel_categories
)

tab_4_columns = pn.Column(
    ("# Averange Sale Price Per Square Foot and Gross Rent in San Francisco"),
    neighborhood_map
)

# Create dashboard Tabs
panel = pn.Tabs(
    ("Info", info),
    ("Welcome",tab_4_columns),
    ("Yearly Rental Analysis", tab_1_columns),
    ("Neighborhood Analysis", tab_2_columns),
    ("Parallel Plots Analysis",tab_3_columns),
    
)

In [7]:
panel

Tabs
    [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01781')
    [2] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01455')
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive01463')
        [5] Markdown(str)
        [6] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive01547')
    [3] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive01634')
                    [0] HoloViews(DynamicMap, name='interactive01634')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive01674')
    [4] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01764')
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01772')

In [8]:
panel.servable()

Tabs(active=4)
    [0] Markdown(str)
    [1] Column
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01781')
    [2] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01455', viewport={'xaxis.range': [2009.5, ...})
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive01463')
        [5] Markdown(str)
        [6] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Curve, name='interactive01547')
    [3] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Row(name='interactive01634')
                    [0] HoloViews(DynamicMap, name='interactive01634')
                    [1] Column
                        [0] WidgetBox
                            [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                        [1] VSpacer()
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] HoloViews(Bars, name='interactive01674')
    [4] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01764')
        [3] Markdown(str)
        [4] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive01772')